In [88]:
import pandas as pd
import numpy as np
from py2neo import Graph
import py2neo
import datetime as dt
from datetime import datetime
pd.set_option('display.max_columns', None)

In [85]:
graph = Graph("bolt://localhost:7687", auth=("neo4j", "test"))

In [3]:
# graph.run("CREATE CONSTRAINT UniqueAirportIdConstraint ON (a:Airport) ASSERT a.id IS UNIQUE").data()
# graph.run("CREATE CONSTRAINT UniqueCityIdConstraint ON (c:City) ASSERT c.id IS UNIQUE").data()
# graph.run("CREATE CONSTRAINT UniqueFlightIdConstraint ON (f:Flight) ASSERT f.id IS UNIQUE").data()
# graph.run("CREATE CONSTRAINT UniqueAirlineIdConstraint ON (al:Airline) ASSERT al.id IS UNIQUE").data()
# graph.run("CREATE CONSTRAINT UniqueStateIdConstraint ON (s:State) ASSERT s.id IS UNIQUE").data()

In [4]:
graph.run("CALL db.constraints()").data()

[{'name': 'UniqueAirlineIdConstraint',
  'description': 'CONSTRAINT ON ( airline:Airline ) ASSERT (airline.id) IS UNIQUE',
  'details': "Constraint( id=8, name='UniqueAirlineIdConstraint', type='UNIQUENESS', schema=(:Airline {id}), ownedIndex=7 )"},
 {'name': 'UniqueAirportIdConstraint',
  'description': 'CONSTRAINT ON ( airport:Airport ) ASSERT (airport.id) IS UNIQUE',
  'details': "Constraint( id=2, name='UniqueAirportIdConstraint', type='UNIQUENESS', schema=(:Airport {id}), ownedIndex=1 )"},
 {'name': 'UniqueCityIdConstraint',
  'description': 'CONSTRAINT ON ( city:City ) ASSERT (city.id) IS UNIQUE',
  'details': "Constraint( id=4, name='UniqueCityIdConstraint', type='UNIQUENESS', schema=(:City {id}), ownedIndex=3 )"},
 {'name': 'UniqueFlightIdConstraint',
  'description': 'CONSTRAINT ON ( flight:Flight ) ASSERT (flight.id) IS UNIQUE',
  'details': "Constraint( id=6, name='UniqueFlightIdConstraint', type='UNIQUENESS', schema=(:Flight {id}), ownedIndex=5 )"},
 {'name': 'UniqueStateId

In [65]:
graph.run("match (n) return count(n)").data()

[{'count(n)': 0}]

In [66]:
"""
Generate flights nodes with timestamps
"""

print(datetime.now().strftime("%H:%M:%S") + '\tStarted')

graph.run('''

    USING PERIODIC COMMIT 500000
    LOAD CSV WITH HEADERS FROM 'file:///2020_4_id.csv' AS row
    WITH row LIMIT 200000000000000
    MERGE (f:Flight {id:toInteger(row.flight_id)})
        ON CREATE SET
              f.plane = row.plane,
              f.airline = row.airline,
              f.flight = row.flight,
              f.origin_airport = toInteger(row.origin_airport),
              f.origin_city = toInteger(row.origin_city),
              f.origin_state = toInteger(row.origin_state),
              f.destination_airport = toInteger(row.destination_airport),
              f.destination_city = toInteger(row.destination_city),
              f.destination_state = toInteger(row.destination_state),
              f.cancelled = row.cancelled,
              f.cancel_code = row.cancel_code,
              f.number_flights = row.number_flights,
              f.distance = toInteger(row.distance),
              f.scheduled_travel_time = toInteger(row.scheduled_travel_time),
              f.year = toInteger(row.year),
              f.month = toInteger(row.month),
              f.day = toInteger(row.day),
              f.departure_scheduled_hour = toInteger(row.departure_scheduled_hour),
              f.departure_scheduled_minute = toInteger(row.departure_scheduled_minute),
              f.arrival_scheduled_hour = toInteger(row.arrival_scheduled_hour),
              f.arrival_scheduled_minute = toInteger(row.arrival_scheduled_minute),
              f.departure_timestamp = datetime({year:f.year,month:f.month, day:f.day, hour:f.departure_scheduled_hour, minute:f.departure_scheduled_minute}),
              f.arrival_timestamp = datetime({year:f.year,month:f.month, day:f.day, hour:f.arrival_scheduled_hour, minute:f.arrival_scheduled_minute})
              '''                 
    )

print(f'Flights created: {graph.run("match (f:Flight) return count(f)").data()}')


print(datetime.now().strftime("%H:%M:%S") + '\tFinished')

17:15:36	Started
Flights created: [{'count(f)': 313382}]
17:16:01	Finished


In [67]:
"""
Generate airports nodes
"""

print(datetime.now().strftime("%H:%M:%S") + '\tStarted')

graph.run('''

    LOAD CSV WITH HEADERS FROM 'file:///airports.csv' AS row
    MERGE (a:Airport {id:toInteger(row.Code)})
        ON CREATE SET
            a.name = row.name,
            a.city = row.city_state,      
            a.description = row.Description
            '''
    )

print(f'Airports created: {graph.run("match (a:Airport) return count(a)").data()}')

print(datetime.now().strftime("%H:%M:%S") + '\tFinished')

17:16:01	Started
Airports created: [{'count(a)': 6545}]
17:16:01	Finished


In [68]:
"""
Generate city nodes
"""

print(datetime.now().strftime("%H:%M:%S") + '\tStarted')

graph.run('''

    LOAD CSV WITH HEADERS FROM 'file:///cities.csv' AS row
    MERGE (c:City {id:toInteger(row.Code)})
        ON CREATE SET
            c.name = row.city,
            c.state = toString(row.state),
            c.description = toString(row.Description)
            '''
    )

print(f'Cities created: {graph.run("match (c:City) return count(c)").data()}')

print(datetime.now().strftime("%H:%M:%S") + '\tFinished')

17:16:01	Started
Cities created: [{'count(c)': 5918}]
17:16:01	Finished


In [69]:
"""
Generate states nodes
"""

print(datetime.now().strftime("%H:%M:%S") + '\tStarted')

graph.run('''

    LOAD CSV WITH HEADERS FROM 'file:///states.csv' AS row
    MERGE (s:State {id:toInteger(row.Code)})
        ON CREATE SET
            s.name = row.Description
            '''
    )

print(f'States created: {graph.run("match (s:State) return count(s)").data()}')

print(datetime.now().strftime("%H:%M:%S") + '\tFinished')

17:16:01	Started
States created: [{'count(s)': 74}]
17:16:01	Finished


### RELATIONSHIPS

In [70]:
"""
Generate flights - airports relationships
F - DEPARTED_FROM_AIRPORT - A
F - ARRIVED_AT_AIRPORT - A
"""

print(datetime.now().strftime("%H:%M:%S") + '\tStarted')



graph.run('''
    MATCH (a:Airport), (f:Flight)
    WHERE f.origin_airport = a.id
    CREATE (f)-[r:DEPARTED_FROM_AIRPORT]->(a)

        ''')
        
    
graph.run('''
    MATCH (a:Airport), (f:Flight)
    WHERE a.id = f.destination_airport
    CREATE (f)-[r:ARRIVED_AT_AIRPORT]->(a)

        ''')
    
print(f'Relationships created: {graph.run("match (:Flight)-[r]->(:Airport) return count(r)").data()}')
    
print(datetime.now().strftime("%H:%M:%S") + '\tFinished')

17:16:01	Started
Relationships created: [{'count(r)': 626764}]
17:16:07	Finished


In [71]:
"""
Generate flights - cities relationships
F - DEPARTED_FROM_CITY - C
F - ARRIVED_AT_CITY - C
"""

print(datetime.now().strftime("%H:%M:%S") + '\tStarted')



graph.run('''
    MATCH (f:Flight), (c:City)
    WHERE f.origin_city = c.id
    CREATE (f)-[r:DEPARTED_FROM_CITY]->(c)

        ''')
        
    
graph.run('''
    MATCH (c:City), (f:Flight)
    WHERE c.id = f.destination_city
    CREATE (f)-[r:ARRIVED_AT_CITY]->(c)

        ''')
    
print(f'Relationships created: {graph.run("match (:Flight)-[r:DEPARTED_FROM_CITY | ARRIVED_AT_CITY]->(:City) return count(r)").data()}')
    
print(datetime.now().strftime("%H:%M:%S") + '\tFinished')

17:16:07	Started
Relationships created: [{'count(r)': 626764}]
17:16:14	Finished


In [72]:
"""
Generate flights - states relationships
F - DEPARTED_FROM_STATE - C
F - ARRIVED_AT_STATE - C
"""

print(datetime.now().strftime("%H:%M:%S") + '\tStarted')



graph.run('''
    MATCH (f:Flight), (s:State)
    WHERE f.origin_state = s.id
    CREATE (f)-[r:DEPARTED_FROM_STATE]->(s)

        ''')
        
    
graph.run('''
    MATCH (s:State), (f:Flight)
    WHERE s.id = f.destination_state
    CREATE (f)-[r:ARRIVED_AT_STATE]->(s)

        ''')
    
print(f'Relationships created: {graph.run("match (:Flight)-[r:DEPARTED_FROM_STATE | ARRIVED_AT_STATE]->(:State) return count(r)").data()}')
    
print(datetime.now().strftime("%H:%M:%S") + '\tFinished')

17:16:14	Started
Relationships created: [{'count(r)': 626764}]
17:16:21	Finished


In [73]:
"""
Generate airports - cities relationships
A - IS_IN - C
"""

print(datetime.now().strftime("%H:%M:%S") + '\tStarted')



graph.run('''
    MATCH (a:Airport), (c:City)
    WHERE a.city = c.description
    CREATE (a)-[r:IS_IN]->(c)
        ''')
        
    
print(f'Relationships created: {graph.run("match (:Airport)-[r]->(:City) return count(r)").data()}')
    
print(datetime.now().strftime("%H:%M:%S") + '\tFinished')

17:16:21	Started
Relationships created: [{'count(r)': 6330}]
17:16:21	Finished


In [74]:
"""
Generate airports - airports relationships
A - IS_IN_SAME_CITY - A
"""

print(datetime.now().strftime("%H:%M:%S") + '\tStarted')



graph.run('''
    MATCH (a:Airport), (a1:Airport)
    WHERE a.city = a1.city
    CREATE (a)-[r:IS_IN_SAME_CITY]->(a1)
        ''')


print(f'Relationships created: {graph.run("match (:Airport)-[r]->(:Airport) return count(r)").data()}')

print(datetime.now().strftime("%H:%M:%S") + '\tFinished')

17:16:21	Started
Relationships created: [{'count(r)': 8475}]
17:16:21	Finished


In [76]:
"""
Generate city - city relationships
S - HAS_FLIGHT_TO - S
"""

print(datetime.now().strftime("%H:%M:%S") + '\tStarted')



graph.run('''
    MATCH (s:State), (s1:State), (f:Flight)
    WHERE f.origin_state = s.id AND f.destination_state = s1.id
    MERGE (s)-[r:HAS_FLIGHT_TO]->(s1)
        ON CREATE SET
            r.datetime = f.departure_timestamp,
            r.distance = f.distance,
            r.time = f.scheduled_travel_time
        ''')
        
    
print(f'Relationships created: {graph.run("match (:State)-[r:HAS_FLIGHT_TO]->(:State) return count(r)").data()}')
    
print(datetime.now().strftime("%H:%M:%S") + '\tFinished')

17:17:08	Started
Relationships created: [{'count(r)': 1175}]
17:17:12	Finished


In [77]:
"""
Generate state - state relationships
C - HAS_FLIGHT_TO - C
"""

print(datetime.now().strftime("%H:%M:%S") + '\tStarted')



graph.run('''
    MATCH (c:City), (c1:City), (f:Flight)
    WHERE f.origin_city = c.id AND f.destination_city = c1.id
    MERGE (c)-[r:HAS_FLIGHT_TO]->(c1)
            ON CREATE SET
            r.datetime = f.departure_timestamp
        ''')
        
    
print(f'Relationships created: {graph.run("match (:City)-[r]->(:City) return count(r)").data()}')
    
print(datetime.now().strftime("%H:%M:%S") + '\tFinished')

17:17:12	Started
Relationships created: [{'count(r)': 4105}]
17:17:16	Finished


Obs: sendo os nodes os voos, a relacao seria a estadia. Assim, para calculo de rotas, a distancia seria o numero de dias na cidade de origem. Possivel?

1. Decide if the trip is world (country wize), continent (country/state wize), country (city/state wize).
2. Select which cities/countries/states to visit based on mandatory requirements
3. Select all possible flights between those cities and calculate all possible paths
4. Score those paths according to optional/preference requirements


## Usecase example:

### Client request
#### Mandatory 
Trip in April for 1 week starting and ending in California, min 2 states and max 3, min 2 days/state, only states started with Mi.
#### Optional
Beach, Casino allowed for <21


### Execution
1. Select set of cities not in Europe.
2. Select set of flights between previous set of cities that have departure_date after July1 and arrival before July31
3. Select all possible paths starting and ending in Lisbon, with [min_cities, max_cities] hops with condition -> next hop dep_time is after current hop land_time
4. Create a list with each path and a point system (for any city with casino allowed +2 points, for every city raining -5 points, etc..
5. Order list and suggest top 5 routes



In [78]:
#Mandatory constants

initial_place = "California"
final_place = 'California'
first_day = dt.datetime(2020, 4,7)
last_day = dt.datetime(2020, 4,15)
days_of_travel = last_day - first_day
min_places = 2
max_places = 2
min_days_per_state = 3
max_days_per_state = 4


In [130]:
#Select list of possible states

possible_states = graph.run('''
    MATCH (s:State)
    WHERE s.name STARTS WITH "M" OR s.name = $initial_place OR s.name = $final_place
    RETURN s.name, s.id
        ''',
    parameters = {
        'final_place': final_place,
        'initial_place': initial_place        
    }
         ).data()

x

['California',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Montana',
 'Marshall Islands (Fips Pub 55-3)',
 'Midway Islands']

In [129]:
# Calculate different routes between states


routes = graph.run('''
    MATCH routes = (s:State)-[r:HAS_FLIGHT_TO*2]->(s1:State)
    WHERE s.name = $initial_place
    AND s1.name = $final_place
    WITH routes, s, s1, r
    RETURN s as state, s1 as state1, r.distance, p
    
        ''',
    parameters = {
        'final_place': final_place,
        'initial_place': initial_place        
    }
         ).data()

print(routes)

ClientError: SyntaxError: Type mismatch: expected Map, Node, Relationship, Point, Duration, Date, Time, LocalTime, LocalDateTime or DateTime but was List<Relationship> (line 6, column 38 (offset: 190))
"    RETURN s as state, s1 as state1, r.distance, p"
                                      ^

In [44]:
test = [list(o.values())[0] for o in routes]

    

(California)-[:HAS_FLIGHT_TO {datetime: datetime('2020-04-01T11:05:00.000000000+00:00')}]->(Washington)-[:HAS_FLIGHT_TO {datetime: datetime('2020-04-01T11:29:00.000000000+00:00')}]->(North Carolina)-[:HAS_FLIGHT_TO {datetime: datetime('2020-04-01T19:59:00.000000000+00:00')}]->(California)

In [ ]:
#Select all the flights in the places available, in the time delta.

flights_list = graph.run('''
    MATCH (f:Flight)
    WHERE f.departure_timestamp > datetime({year:$dep_year,month:$dep_month, day:$dep_day})
    AND f.arrival_timestamp < datetime({year:$arr_year,month:$arr_month, day:$arr_day})
    AND f.origin_state IN $states_ids
    AND f.destination_state IN $states_ids
    RETURN f.id
        ''',
    parameters = {
        'first_day': first_day,
        'last_day': last_day,
        'states_ids': states_ids,
        'dep_year': first_day.year,
        'dep_month': first_day.month,
        'dep_day': first_day.day,
        'arr_year': last_day.year,
        'arr_month': last_day.month,
        'arr_day': last_day.day  
    }
         ).data()

flights_list_ids = [list(o.values())[0] for o in flights_list]
print(f'Flights selected: {len(flights_list_ids)}')

In [ ]:
#     AND f.arrival_timestamp < Datetime({})
#     AND f.arrival_timestamp + Duration({days:$max_days_per_state}) > f1.departure_timestamp

In [ ]:
# Create relationships with possible sequence of states

print(datetime.now().strftime("%H:%M:%S") + '\tStarted')

graph.run('''
    MATCH (f:Flight), (s:State), (s1:State)
    WHERE f.id IN $flights_list_ids
    AND s.id = f.origin_state
    AND s1.id = f.destination_state
    AND f.origin_state <> f.destination_state
    MERGE (s)-[r:HAS_FLIGHT {date: datetime({year:f.year, month:f.month, day:f.day})}]->(s1)
        ON CREATE SET 
            r.distance = toInteger(f.distance),
            r.duration = toInteger(f.scheduled_travel_time)
    

        ''',
    parameters = {
        'flights_list_ids': flights_list_ids,
        'stated_ids': states_ids    
    }
         )

print(f'Relationships created: {graph.run("match (:State)-[r:HAS_FLIGHT]->(:State) return count(r)").data()}')
print(datetime.now().strftime("%H:%M:%S") + '\tFinished')

In [62]:
# Delete entire graph
graph.run('MATCH (n) DETACH DELETE n')
print(graph.run("match (n) return count(n)").data())

[{'count(n)': 0}]


In [ ]:
print(datetime.now().strftime("%H:%M:%S") + '\Started')
print(graph.run('MATCH (f:Flight)-[r:DEPARTED_FROM_AIRPORT]->(a:Airport)-[r2:IS_IN]->(c:City) WHERE c.name = "Chicago" RETURN count(*)').data())
print(datetime.now().strftime("%H:%M:%S") + '\tFinished')

In [ ]:
print(datetime.now().strftime("%H:%M:%S") + '\Started')
print(graph.run('MATCH (f:Flight)-[r:DEPARTED_FROM_CITY]->(c:City) WHERE c.name = "Chicago" RETURN count(*)').data())
print(datetime.now().strftime("%H:%M:%S") + '\tFinished')

In [ ]:
call semantics.mapping.addSchema("http://schema.org/", "Person", "Person")
call semantics.mapping.addSchema("http://schema.org/Person#", "knows", "knows")
call semantics.mapping.addSchema("http://schema.org/Person#", "friendOf", "colleagues")

minimum spanning tree -  an algo that 'touches' all the nodes in the shortest way possible
strongly connected components - an algo to detect nodes or groups of nodes highly connected